In [2]:
import numpy as np

x = np.array([None])
print(x[0])

None


In [13]:
numVar = 5

# -- Entrada: Una matriz simplex de las restricciones
# Si es max entonces estandarizamos y multiplicamos el renglon de costos por -1 y agregamos a la matrix simplex
# Agregamos variables de holgura y excedente
# Checar si contiene la identidad, hay soluciones degeneradas, checar si esta acotado o no

# Para renglon de costos
costos = [5,4,1,9,-2]
renglonCostos = np.zeros(numVar+1)
for i in range(numVar):
    renglonCostos[i] = costos[i]


# Para restricciones
rest1 = [4,0,0,3,1,10,2] # ultimo indice de 0 a 2 representando <=, >= y =
renglonRest1 = np.zeros(numVar+1)
for i in range(numVar+1):
    renglonRest1[i] = rest1[i]
    



#matrix = np.vstack((arr2, arr3, arr4))
#arr1 = np.array([[9,9,9]]).T
arr1 = np.array([np.ones(3)]).T
print(arr1[0:2])
#mat2 = np.vstack((mat[0:2], arr1, mat[2:,:])) # Insertar un renglon
mat2 = np.hstack((mat[:,0:2], arr1, mat[:,2:])) # Insertar una columna
print(mat)
print()
print(mat2)
print()

#print((mat[1,:]/mat[2,:])[2] == float('inf'))

[[1.]
 [1.]]


NameError: name 'mat' is not defined

In [ ]:
arrMin = np.array([1,1,3,4,-1])

print(arrMin.min())
ind = np.where(arrMin == 3)
print(ind)

print(ind[0][0])

-1
(array([2]),)
2


In [ ]:
def checkNegativos(arr: np.ndarray):
    # Regresa -1 si ninguno es negativo, si no regresa el indice del primer elemento negativo
    ind = 0
    for e in arr:
        if e < 0:
            return ind
        ind += 1
    return -1

def checkRenglonPivote(y: np.ndarray, b: np.ndarray):
    arrLength = b.size
    epsilon = np.ones(arrLength) # Crea un vector donde guardaremos todas las epsilon=bi/yi
    for i in range(arrLength):
        yi, bi = y[i], b[i]
        if yi == 0 or bi < 0 or yi < 0: 
            # Si yi = 0, o bi,yi < 0 entonces invalidamos esos cocientes guardando -infinito en esa entrada del
            # vector epsilon
            epsilon[i] = float('inf')
        else:
            # Si bi y yi son validos guardamos su cociente
            epsilon[i] = bi/yi
            
    minEpsilon = epsilon.min() # Encontramos el mínimo de epsilon
    if minEpsilon == float('inf'):
        raise Exception("No existe epsilon valida")
    else:
        indiceMinEpsilon = np.where(epsilon == minEpsilon)[0][0] # Si existen dos epsilons minimos iguales agarramos el primero (Regla de Bland)
    
    return indiceMinEpsilon

def simplex(tabla: np.ndarray):
    # Entra una tabla simplex estandar inicial y sale la tabla simplex final
    costos = tabla[-1,:]
    numReng = tabla.shape[0]
    numCol = tabla.shape[1]
    
    print('--------------------------------------------------------------\n')
    
    while checkNegativos(costos) != -1:
        # Vemos que el indice de la variable que va a entrar (Siempre entra el más a la izquierda Regla de Bland)
        indVarEntrada = checkNegativos(costos)
        # Determinamos el indice del renglon que va a ser pivote
        indRengPivote = checkRenglonPivote(tabla[:,indVarEntrada], tabla[:,-1])
        
        print("Entra la variable: ", indVarEntrada+1)
        print("Pivoteamos el renglón: ", indRengPivote+1)
        print(tabla)
        
        # Hacemos ese primer 1 pivote
        tabla[indRengPivote,:] = tabla[indRengPivote,:]/tabla[indRengPivote,indVarEntrada]
        
        # La suma de renglones
        for i in range(numReng):
            if i != indRengPivote:
                tabla[i,:] = tabla[i,:] - (tabla[i,indVarEntrada])*tabla[indRengPivote, :]
    
        print('--------------------------------------------------------------\n')

    print("Tabla Final")
    print(tabla)
        
    

In [ ]:
mat = np.array([[0,5,50,1,1,0,10],[1,-15,2,0,0,0,2],[0,1,1,0,1,1,6],[0,-10,-2,0,1,0,-6]]).astype(float)
simplex(mat)

--------------------------------------------------------------

Entra la variable:  2
Pivoteamos el renglón:  1
[[  0.   5.  50.   1.   1.   0.  10.]
 [  1. -15.   2.   0.   0.   0.   2.]
 [  0.   1.   1.   0.   1.   1.   6.]
 [  0. -10.  -2.   0.   1.   0.  -6.]]
--------------------------------------------------------------

Tabla Final
[[  0.    1.   10.    0.2   0.2   0.    2. ]
 [  1.    0.  152.    3.    3.    0.   32. ]
 [  0.    0.   -9.   -0.2   0.8   1.    4. ]
 [  0.    0.   98.    2.    3.    0.   14. ]]


In [ ]:
def estandarizar(restricciones: np.ndarray, costos: np.ndarray, isMin: bool, nomVariables: list = []):
    costos = np.array([np.insert(costos, -1, 0)])
    restricciones = np.vstack((restricciones, costos))
    numRest = restricciones.shape[0]
    
    numVarsH = 1
    numVarsE = 1
    
    if not isMin:
        costos = -1*costos
        
    # La idea es que cuando pasemos las restricciones sean arreglos donde los primeros n elementos
    # sean las n variables de esa restricción, el elemento (n+1) sea la (b) de esa restricción y
    # que el ultimo elemento sea -1 si la restricción es <=, 1 si es >= y 0 si es =
    
    for i in range(numRest):
        op = restricciones[i,-1]
        if op == -1: # <=
            # Agregar variable de holgura
            colH = np.zeros(numRest) 
            colH[i] = 1 # Creamos columna de ceros menos un 1 en el current renglon 
            colH = np.array([colH]).T # Transponemos para hacer arreglo en columna
            restricciones = np.hstack((restricciones[:,0:-2], colH, restricciones[:,-2:]))
            nomVariables.append("h"+str(numVarsH))
            numVarsH += 1
        elif op == 1: # >=
            # Agregar variable de excedente
            colE = np.zeros(numRest) 
            colE[i] = -1 # Creamos columna de ceros menos un 1 en el current renglon 
            colE = np.array([colE]).T # Transponemos para hacer arreglo en columna
            restricciones = np.hstack((restricciones[:,0:-2], colE, restricciones[:,-2:]))
            nomVariables.append("e"+str(numVarsE))
            numVarsE += 1
        elif op != 0: # =
            print("ERROR: En creación de variables de holgura y/o excedente")
    
    tablaSimplexStd = restricciones[:,0:-1] # Quitamos la ultima columna que solo representaba las operaciones
    return tablaSimplexStd, nomVariables
    
            
    

In [15]:
matNEst = np.array([[1,2,10,-1],[1,2,5,-1],[1,-1,2,-1]])
costNEst = np.array([[2,-1,0]])
nomVar = ["x1","x2"]

#print(costNEst)
# print(estandarizar(matNEst, costNEst, False, nomVar)[0])
print(matNEst[0:-1,:])

[[ 1  2 10 -1]
 [ 1  2  5 -1]]


In [56]:
import re
def parseFuncionObjetivo(funcObj: str):
    funcObj = funcObj.replace(" ", "").replace("-x","-1x").replace("+","")
    pattern = re.compile(r'x\d')
    funcArr = pattern.split(funcObj)[:-1]

    rengCostos = np.zeros(len(funcArr))
    for i in range(len(funcArr)):
        rengCostos[i] = float(funcArr[i])
    
    return funcArr

In [58]:
funcObj = "3x1 +  6x2 - x3 + 2x4"
print(parseFuncionObjetivo(funcObj))

['3', '6', '-1', '2']


In [9]:
ar = np.array([np.zeros(7)])
print(ar.size)

print("Hello" + str(1))

7
Hello1
